In [35]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/src")
from importlib import reload
import utils
reload(utils)
from utils import *

In [39]:
X_train_r = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/database/model_input/X_train_val_rekriged.csv').set_index('fire_id')
y_train_r = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/database/model_input/y_train_val_rekriged.csv').set_index(X_train_r.index)
y_train_r.drop(columns=y_train_r.columns[0], inplace=True)
X_train_r.drop(columns=X_train_r.columns[0], inplace=True)

In [41]:
y_train_r.isna().sum()

target_encoded    0
dtype: int64

In [21]:
df  =pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/database/model_input/cleaned_final_rekriged.csv')

In [9]:
from torch.utils.tensorboard import SummaryWriter
import numpy as np

writer = SummaryWriter('runs/mnist_experiment_1')

for n_iter in range(100):
    writer.add_scalar('Loss/train', np.random.random(), n_iter)
    writer.add_scalar('Loss/test', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/train', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/test', np.random.random(), n_iter)

---

In [ ]:
dic = {'Pianura': 0,
       'Collina': 1,
       'Montagna': 2}
gdf['zalt'] = gdf['zalt_des'].map(dic)

In [ ]:
gdf.drop(columns=['cod_istat', 'zalt_des'], inplace=True)
gdf.rename(columns={'zalt': 'zona_altimetrica'}, inplace=True)

In [ ]:
dic = dict.fromkeys([col for col in gdf.columns if col not in ['geometry']])
for a in dic:
    dic[a] = 'ALTIMETRIA_'+a

gdf.rename(columns=dic, inplace=True)

In [ ]:
gdf.to_file('/nfs/home/genovese/thesis-wildfire-genovese/database/altimetria.geojson', driver='GeoJSON', index=False)

---

In [ ]:
mising = df[df.height.isna()]
# 45 points

In [ ]:
# got from https://twcc.fr/
fill = {1: 1580,
 125: 569,
 131: 1049,
 146: 1447,
 165: 1345,
 217: 1530,
 225: 1339,
 227: 1361,
 230: 1389,
 258: 563,
 274: 614,
 351: 248,
 383: 642,
 394: 725,
 396: 1298,
 397: 1536,
 404: 766,
 438: 1093,
 440: 1034,
 443: 1356,
 448: 1053,
 458: 1494,
 460: 1298,
 461: 1029,
 468: 559,
 476: 1428,
 477: 1769,
 482: 1538,
 484: 1436,
 501: 406,
 504: 891,
 634: 344,
 635: 373,
 760: 428,
 772: 891,
 826: 1039,
 866: 533,
 996: 1068,
 1011: 568,
 1012: 568,
 1025: 812,
 1035: 585,
 1148: 742,
 1174: 1128,
 1206: 923}

In [ ]:
df['height'].fillna(fill, inplace=True)

In [ ]:
save_clean_data(df, 'kriging_weather_grid', '/nfs/home/genovese/thesis-wildfire-genovese/data/data_loader_for_kriging' )